In [ ]:
%pip install nibabel numpy matplotli opencv-contrib-python scikit-image pydicom

In [ ]:
from __future__ import print_function
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import os
import random

https://stackoverflow.com/questions/61833375/loading-a-nifti-through-nibabel-and-using-the-shape-function

In [ ]:
def get_slice(nii_data, slice):
  return nii_data[:,:,slice]

In [ ]:
def open_nii(path):
  my_img  = nib.load(path)
  nii_data = my_img.get_fdata()
  return nii_data

In [ ]:
def show_slice(nii_data, slice):
  plt.imshow(get_slice(nii_data, slice))
  plt.show()

In [ ]:
def show(img):
  plt.imshow(img)
  plt.show()

In [ ]:
def show_all_slices(nii_data):
  for slice in range(nii_data.shape[2]):
    plt.imshow(get_slice(nii_data,slice))
    plt.show()

In [ ]:
def show_with_label(img, label):
  plt.figure(figsize = (50,7))
  plt.subplot(1,2,1)
  plt.imshow(img, 'gray', interpolation='none')
  plt.imshow(label, 'jet', interpolation='none', alpha=0.1)
  plt.show()

In [ ]:
def cv_convert(img):
  return cv.normalize(img, None, 0, 255, cv.NORM_MINMAX, cv.CV_8U)

In [ ]:
def is_point_in_mask(mask, x, y):
    # Check if the point is within the mask boundaries
    if 0 <= x < mask.shape[1] and 0 <= y < mask.shape[0]:
        # Check if the pixel value at the specified coordinates is non-zero
        return mask[y, x] == 2 # 0 is background, 1 is pancreas, 2 is cancer
    else:
        # Point is outside the mask boundaries
        return False


In [ ]:
def preprocess(ctr):
  img_dir = "../data/Task07_Pancreas/imagesTr"
  files = os.listdir(img_dir)
  label_dir = "../data/Task07_Pancreas/labelsTr"

  for file in files:
    num = file.split(".")
    n = num[0].split("_")
    if len(n) < 2:
      print(n)
      continue
    img_path = os.path.join(img_dir, f"pancreas_{n[1]}.nii.gz")
    label_path = os.path.join(label_dir, f"pancreas_{n[1]}.nii.gz")
    # check if there is cancer in label
    img_nii = open_nii(img_path)
    label_nii = open_nii(label_path)
    
    tumor_slices = []
    
    for i in range(0, label_nii.shape[2]-1):
      label_slice = get_slice(label_nii, i)
      if any(2 in nested_list for nested_list in label_slice):
        tumor_slices.append(i)

    if len(tumor_slices) == 0:
      continue

    chosen = random.choice(tumor_slices)
    # loop through slices that have cancer in label

    img = cv_convert(get_slice(img_nii, chosen))
    label = get_slice(label_nii, chosen)
    top = 10000
    left = 10000
    right = 0
    bottom = 0

    for i, row in enumerate(label):
      for j, pixel in enumerate(row):
        if pixel == 0:
          continue
        if i < top:
          top = i
        if j < left:
          left = j
        if i > bottom:
          bottom = i
        if j > right:
          right = j
    
    cv.imwrite(f"../data/preprocessed_class/tumor/{n[1]}_{chosen}.png", img[top-1:bottom+2, left-1:right+2])
    ctr = ctr -1
    if ctr == 0:
      break


In [ ]:
preprocess(240)

In [ ]:
#np.set_printoptions(threshold=np.inf)

In [ ]:
import os
import pydicom
import numpy as np
import nibabel as nib
from skimage import exposure
from skimage import io
import cv2 as cv
import matplotlib.pyplot as plt
import random

# Function to load a DICOM image
def load_dicom_image(file_path):
    dicom_data = pydicom.dcmread(file_path)
    image_array = dicom_data.pixel_array.transpose(1,0)
    return cv.normalize(image_array, None, 0, 255, cv.NORM_MINMAX, cv.CV_8U)

# Function to load the label data
def load_label_data(label_path):
    label_image = nib.load(label_path)
    label_array = label_image.get_fdata()
    return label_array

def show_with_label(img, label):
  plt.figure(figsize = (50,7))
  plt.subplot(1,2,1)
  plt.imshow(img, 'gray', interpolation='none')
  plt.imshow(label, 'jet', interpolation='none', alpha=0.1)
  plt.show()

# Function to crop the image to the size of the pancreas
def crop_to_pancreas(image, label):
    top = 10000
    left = 10000
    right = 0
    bottom = 0

    for i, row in enumerate(label):
      for j, pixel in enumerate(row):
        if pixel == 0:
          continue
        if i < top:
          top = i
        if j < left:
          left = j
        if i > bottom:
          bottom = i
        if j > right:
          right = j
    
    return image[top-1:bottom+2, left-1:right+2]

# Function to preprocess and save the image as a PNG file
def preprocess_and_save_image(image_path, label_path, save_folder, slc):
    # Load DICOM image
    dicom_image = load_dicom_image(image_path)
    
    # Load label data
    label_data = load_label_data(label_path)

    if not any(1 in nested_list for nested_list in label_data):
      print("empty")
      return 0
    
    # Crop to pancreas
    cropped_image = crop_to_pancreas(dicom_image, label_data[:,:,slc])
    if len(cropped_image) == 0:
      return 0
    
    # Save the cropped image as a PNG file
    save_path = os.path.join(save_folder, f"{slc}-{os.path.basename(label_path).replace('.nii', '.png')}")
    cv.imwrite(save_path, cropped_image)
    return 1

# Set the paths to the dataset and labels
dataset_path = "../data/archive/Pancreas-CT/Pancreas-CT"
label_path = "../data/archive/TCIA_pancreas_labels-02-05-2017/TCIA_pancreas_labels-02-05-2017"
save_folder = "../data/preprocessed_class/pancreas"

# Create the save folder if it doesn't exist
os.makedirs(save_folder, exist_ok=True)

# Loop through all DICOM images in the dataset recursively
for root, dirs, files in os.walk(dataset_path):
      cnt = 0
      for file_name in files:
          if file_name.endswith(".dcm"):
              image_path = os.path.join(root, file_name)
              splits = root.split("/")
              num = splits[5].split("_")[1]
              label_file = os.path.join(label_path, f"label{num}.nii")
              cnt = cnt + preprocess_and_save_image(image_path, label_file, save_folder, int(file_name.split("-")[1].split(".")[0])-1)
              if cnt >=3:
                break


print("Preprocessing completed.")
